In [58]:
import numpy as np
import pandas as pd

In [59]:
X_train = pd.read_csv("../Data/train.csv")
df_macro = pd.read_csv("../Data/macro.csv")

In [60]:
y_train = X_train[["price_doc"]]
X_train = X_train.drop("price_doc", axis = 1)

In [61]:
X_train_sample = X_train[["full_sq", "life_sq","floor"]].fillna(0).sample(5000)
y_train_sample = y_train.loc[X_train_sample.index]

In [62]:
y_train_sample = y_train_sample/1000000

In [65]:
df_train = pd.read_csv("../Data/df_train_without_dummies")
df_train_dummies = pd.read_csv("../Data/df_train_dummies")

In [67]:
df_full = df_train.merge(df_train_dummies)

In [69]:
col_added = ['num_room',
 'full_sq',
 'sadovoe_km',
 'deposits_value',
 'state_4.0',
 'bulvar_ring_km',
 'sport_count_3000',
 'unemployment',
 'floor',
 'big_market_km',
 'build_count_before_1920',
 'life_sq',
 'big_church_count_2000',
 'sport_count_500',
 'mosque_count_5000',
 'cpi',
 'cafe_count_1000_price_high',
 'state_2.0',
 'prom_part_3000',
 'invest_fixed_capital_per_cap',
 'nuclear_reactor_km',
 'green_part_2000',
 'cafe_count_5000_price_2500',
 'material_4.0',
 'cafe_count_500_price_high',
 'cafe_avg_price_5000',
 'max_floor',
 'mkad_km',
 'material_6.0',
 'cafe_count_2000',
 'big_church_count_3000',
 'office_sqm_1500',
 'university_km',
 '0_17_female',
 'cafe_count_2000_price_4000',
 'leisure_count_500',
 'industrial_km',
 'school_quota',
 '0_17_male',
 'church_count_3000',
 'students_state_oneshift',
 'pop_total_inc',
 'heating_share',
 'gdp_deflator',
 'ttk_km',
 'school_education_centers_raion',
 'prom_part_1500',
 'office_count_5000',
 '0_13_all',
 'fitness_km']

In [77]:
X_train = df_full[col_added]

In [101]:
y_train = pd.read_csv("../Data/train.csv")[["price_doc"]].loc[X_train.index]

In [102]:
X_train_normalize = (X_train - X_train.mean())/X_train.std()

In [103]:
y_train_normalize = (y_train - y_train.mean())/y_train.std()

In [119]:
X_train_normalize.shape

(27424, 50)

## Neural Network

In [117]:
class NeuralNetwork():
    def fit(self, X, y, n_hidden, nodes, activations, lr):
        self._lr = lr
        self._X = X.values
        self._y = y.values
        self._n_hidden = n_hidden
        self._nodes = nodes
        self._weights = self._generate_weights()
        self._biases = self._generate_bias()
        self._activations = activations
        self._forward_inputs = []
        
        self._train()
        
    def _activation(self, data, activation = "relu"):
        if activation == "relu":
            def relu(data):
                return np.array([max(0,i) for i in data]).reshape(data.shape)
            return np.apply_along_axis(relu, 1, data)
        if activation == "sigmoid":
            def sigmoid(data):
                return (1/(1 + np.exp(-data))).reshape(data.shape)
            return np.apply_along_axis(sigmoid, 1, data)
    
    def _der_activation(self, points, activation = "relu"):
        if activation == "relu":
            def d_relu(point):
                return np.array([0 if y <= 0 else 1 for y in point])
            return np.apply_along_axis(d_relu, 1, points)
        if activation == "sigmoid":
            ## todo
            return
    
    def _loss_function(self, ypred, loss = "l2"):
        if loss == "mse":
            return ((ypred - self._y) ** 2).mean()
        if loss == "l2":
            return (((ypred - self._y) ** 2)/2).mean()
    
    def _loss_jacobian(self, ypred, loss = "l2"):
        if loss == "l2":
            return (ypred - self._y)/(len(ypred))
    
    def _generate_weights(self):
        hidden_weights = []
        nodes = self._nodes
        for idx in range(1,len(nodes)):
            hidden_weights.append(0.01 *np.random.randn(nodes[idx -1], nodes[idx]))

        return hidden_weights
    
    def _generate_bias(self):
        hidden_layers = []
        nodes = self._nodes
        for i in range(self._n_hidden + 1):
            hidden_layers.append(np.zeros((nodes[i + 1], 1)))
        return hidden_layers
    
    def _forward_propagation(self):
        """
        Suppose 2 observations
        
        Suppose previous layer is 3 nodes
        Suppose current layer is 2 nodes
        
        prev shape (2,3)
        prev = ob1 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               ob2 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               
        layer shape (3,2)
        layer = [weight for current_node_1 for prev_node_1, weight for current_node_2 for prev_node_1]
                [weight for current_node_1 for prev_node_2, weight for current_node_2 for prev_node_2]
                [weight for current_node_1 for prev_node_3, weight for current_node_2 for prev_node_3]
                
        output shape (2,2) # since 2 observations and 2 layers
        output = ob1 [current_node_1 val, current_node_2 val]
                 ob2 [current_node_1 val, current_node_2 val]
                 
        Then for bias in current layer it is (2,1) since 2 nodes in current layer
        
        So for each row in output we add the bias row wise and apply the activation function to each row
        
        prev <- ouput
        
        Move onto next layer...
        """
        prev = self._X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                self._forward_inputs.append((prev, None))
                prev = (prev @ layer) + biases[idx].T,
            else:
                weight_output = (prev @ layer) + biases[idx].T
                self._forward_inputs.append((prev, weight_output))
                prev = self._activation(data = weight_output, activation = activations[idx])

        return prev
    
    def _backward_propagation(self, ypred):
        j = self._loss_jacobian(ypred)
                
        for i in range(len(self._forward_inputs)-1, -1, -1):
            if i != (len(self._forward_inputs) - 1):
                # activation func on all layers except the last
                der_acti = self._der_activation(self._forward_inputs[i][1])
                j = np.multiply(j,der_acti)

            x = self._forward_inputs[i][0]

            jw = x.T.dot(j)

            b = np.ones((j.shape[0],1))
            jb = j.T.dot(b)
            
            self._weights[i] -= self._lr * jw
            self._biases[i] -= self._lr * jb

            j = j.dot(self._weights[i].T)
            
        self._forward_inputs = []
        
    
    def _train(self):
        for i in range(0, 100):
            out = self._forward_propagation()
            loss = self._loss_function(out[0])
            mse = self._loss_function(out[0], loss = "mse")
            print("\nloss:")
            print(self._loss_function(out[0]))
            print("rmse:")
            print(mse)
            #print("\npredictions\n")
            #print(out)
            self._backward_propagation(out[0])

In [118]:
INPUT_SIZE = X_train.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 1
nodes = [INPUT_SIZE,50,50,OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]

nn = NeuralNetwork()

nn.fit(X = X_train_normalize,
       y = y_train_normalize,
       n_hidden = len(nodes) - 2,
       nodes = nodes,
       activations = activations,
       lr = LEARNING_RATE)


loss:
0.4999764278706508
nmse:
0.9999528557413015

loss:
0.49997545754010586
nmse:
0.9999509150802117

loss:
0.4999745143242498
nmse:
0.9999490286484995

loss:
0.4999736070890656
nmse:
0.9999472141781311


KeyboardInterrupt: 